In [63]:
''' My Useful Functions
'''

import unicodedata
import numpy as np
import requests
import glob, os
from bs4 import BeautifulSoup

# 1. Tries to download page 6 times instead of 1
def long_request(url):
    for j in range(6):
        try:
            p = requests.get(url)
            return p
            #break
        except Exception as e:
            print(e)
    return 'CAN_NOT_CONNECT'

# 2. Checks if list has the same elements
def Doubles(a):
    ans = []
    for i in range(1,len(a)):
        if a[i] in set(a[:i]):
            ans.append(a[i])
    return ans

# 3. Checks if a table has columns or rows with the same name.(That's a huge mistake!)
def DRDC(table):
    a = ['raws']
    a.extend(Doubles(table.index))
    a.append('columns')
    a.extend(Doubles(table.columns))
    return a

# 4. Delete accents above players names
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

# 5. strip_accents for the 1st clumn of the table
# Need to be DataFrame,not a column constti.strip_accents_pdlist(pd.DataFrame(bigTable['web_name']))
def strip_accents_pdlist(a):
    b = a.copy()
    for i in range(len(a)):
        b.iat[i,0] = strip_accents(a.iat[i,0])
    return b

# 6. Changes name of a column col_name to new_col_name of a Table
def change_column_name(Table, col_name, new_col_name):
    if new_col_name in set(Table.columns):
        print("Can't change column name")
        return Table
    temp_Table = Table.copy()
    temp = temp_Table.columns
    temp = [temp[i] if temp[i] != col_name else new_col_name for i in range(len(temp))]
    temp_Table.columns = temp
    return temp_Table

# 7. Finds differences between the tables

def differences(A, B):
    '''
        Finds differences between the tables
        Change None to np.nan previously
    '''
    same = 1
    if list(A.index) != list(B.index):
        print(f'Indexes are not equal: {(set(A.index)-set(B.index))|(set(B.index)-set(A.index))}')
        same = 0
    if list(A.columns) != list(B.columns):
        print(f'Columns are not equal: {(set(A.columns)-set(B.columns))|(set(B.columns)-set(A.columns))}')
        same = 0
    if same == 0:
        return A
    else:
        A = A.fillna(value=np.nan)
        B = B.fillna(value=np.nan)
        def eqNeq(col):
            res = col.copy()
            for i in A.index:
                a = type(col[i]) not in (float, np.float64)
                b = type(B[col.name][i]) not in (float, np.float64)
                if a|b: res[i] = bool(col[i] == B[col.name][i])
                else:
                    c = np.isnan(col[i])
                    d = np.isnan(B[col.name][i])
                    if c&d: res[i] = True
                    else: res[i] = bool(col[i] == B[col.name][i])
            return res
        df = A.apply(eqNeq)
#         df = A.apply(lambda x: [x[i] == B[x.name][i] \
#         if (type(x[i]) not in (float, np.float64))|(type(B[x.name][i]) not in (float, np.float64))\
#         else True if (np.isnan(x[i]))&(np.isnan(B[x.name][i]))\
#         else x[i] == B[x.name][i] for i in A.index])

        for i in df.index:
            if df.loc[i].sum() == len(df.columns):
                df = df.drop(i)
        for j in df.columns:
            if df[j].sum() == len(df):
                del df[j]
        return df
    
# 8. isnan without mistake for non numpy objects

def myisnan(x):
    try:
        return np.isnan(x)
    except:
        return False

# 9. Same head, header and script at the end for all html files inside html folder

def head_header_script (copy_head = 1, copy_header = 1, copy_script = 1, copy_nav=1, copy_footer=1):
    '''
        Same head, header and script at the end for all html files inside html folder
    '''
    with open('index.html', 'r', encoding="utf-8") as file:
        main_html = file.read()
        header = str(BeautifulSoup(main_html, 'html.parser')('header')[0])
        head = str(BeautifulSoup(main_html, 'html.parser')('head')[0])
        script = str(BeautifulSoup(main_html, 'html.parser')('script')[-1])
        nav = str(BeautifulSoup(main_html, 'html.parser')('nav')[0])
        footer = str(BeautifulSoup(main_html, 'html.parser')('footer')[0])
        

    my_path = os.getcwd()
    file_paths = glob.glob(my_path + '/html/**/*.html', recursive=True)

    for path in file_paths:
        with open(path, 'r', encoding="utf-8") as file:
            print(path)
            file_text = file.read()
        
        new_file = file_text
        if (file_text.find('<head>') != -1) & (copy_head == 1):
            new_file = file_text.replace(file_text[file_text.find('<head>'):file_text.find('</head>')+7], head)
        if (file_text.find('<header>') != -1) & (copy_header == 1):
            new_file = new_file.replace(new_file[new_file.find('<header>'):new_file.find('</header>')+9], header)
        if (new_file.find('<script id="end">') != -1) & (copy_script == 1):
            #print("found!!")
            new_file = new_file.replace(new_file[new_file.find('<script id="end">'):], script)
        if (file_text.find('<nav>') != -1) & (copy_nav == 1):
            new_file = new_file.replace(new_file[new_file.find('<nav>'):new_file.find('</nav>')+6], nav)
        if (file_text.find('<footer>') != -1) & (copy_nav == 1):
            new_file = new_file.replace(new_file[new_file.find('<footer>'):new_file.find('</footer>')+9], footer)

#             file_text1 = str(BeautifulSoup(file_text, 'html.parser')('div', {id:"main-table"})[0])
#             file_text2 = str(BeautifulSoup(file_text, 'html.parser')('div', {id:"blog"})[0])
#             file_text = file_text1 + file_text2
#             new_file = head + "<body>" + header + nav + file_text + "</body>" + footer + script
        
        #else:
        #Check if it i not a Data page
        if ('FPL' not in path.split('\\')) and ('Understat' not in path.split('\\')):
            #print("not found")
            div_sources = str(BeautifulSoup(main_html, 'html.parser')('div', {"id" : "Sources"})[0])
            new_file = head + "<body>" + header + "<main>" + file_text + "</main>" + "</body>" + footer + script
            new_file = new_file.replace(div_sources, "")
        with open(path, 'w', encoding="utf-8") as file:
            file.write(new_file)
            
#     file_paths = glob.glob(my_path + '/Ranking/**/*.html', recursive=True)
#     for path in file_paths:
#         with open(path, 'r', encoding="utf-8") as file:
#             print(path)
#             file_text = file.read()
#         new_file = head + header + file_text + footer + script
#         with open(path, 'w', encoding="utf-8") as file:
#             file.write(new_file)

if __name__=="__main__":
    print('Hello')
    p = long_request('http://google.com')
    print(p.text)

Hello
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ru"><head><meta content="&#1055;&#1086;&#1080;&#1089;&#1082; &#1080;&#1085;&#1092;&#1086;&#1088;&#1084;&#1072;&#1094;&#1080;&#1080; &#1074; &#1080;&#1085;&#1090;&#1077;&#1088;&#1085;&#1077;&#1090;&#1077;: &#1074;&#1077;&#1073; &#1089;&#1090;&#1088;&#1072;&#1085;&#1080;&#1094;&#1099;, &#1082;&#1072;&#1088;&#1090;&#1080;&#1085;&#1082;&#1080;, &#1074;&#1080;&#1076;&#1077;&#1086; &#1080; &#1084;&#1085;&#1086;&#1075;&#1086;&#1077; &#1076;&#1088;&#1091;&#1075;&#1086;&#1077;." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="9Ep0wsPwCvmf4r+gHJVhlQ==">(function(){window.google={kEI:'TjfzYYTxLcOG9u8PguWjsAs',kEXPI:'0,18168,1284368,56873,6058,207,4804,2316,383,246,5,1354,4013,1238,1122515,1197713,688,380089,16115,2

In [64]:
head_header_script()

C:\Users\const\Documents\GitHub\tikeve.github.io/html\2019_2020\index.html


IndexError: list index out of range

In [58]:
path.split('\\')

['C:',
 'Users',
 'const',
 'Documents',
 'GitHub',
 'tikeve.github.io/html',
 'Understat',
 'Player_xG.html']

In [60]:
('FPL' not in path.split('\\')) and ('Understat' not in path.split('\\'))

False